In [5]:
import re
import fitz  
import json
import pickle
import numpy as np
import faiss
from pathlib import Path
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [6]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def clean_text(text):
    text = re.sub(r'Pagina?\s+\d+\s+din\s+\d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\d+/\d+', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'\d{2}\.\d{2}\.\d{4}', '', text)
    
    # Normalizează spațiile
    text = re.sub(r'\s{3,}', ' ', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    return text.strip()

def identify_doc_type(filename):
    filename = filename.lower()
    if 'codul-muncii' in filename or 'cod muncii' in filename:
        return 'CM'
    elif 'constitutia' in filename:
        return 'CONST'
    elif 'cod-civil' in filename or 'cod civil' in filename:
        return 'CC'
    elif 'cod-fiscal' in filename or 'cod fiscal' in filename:
        return 'CF'
    else:
        return 'LEGAL'

def is_article_reference(text_line):

    reference_patterns = [
        r'^\s*(?:conform|potrivit|prevederile|prevazut\s+(?:la|de)|dispozitiile|in\s+sensul|in\s+conditiile)\s+(?:art\.|articolul)\s+\d+',
        r'(?:conform|potrivit|prevederile|prevazut\s+(?:la|de)|dispozitiile)\s+(?:art\.|articolul)\s+\d+(?:\s*[-,.]|\s*$)',
        r'^\s*(?:art\.|articolul)\s+\d+\s*[-,.]?\s*(?:alin|lit|pct)\.',  # art. 132 alin. 1
        r'la\s+(?:art\.|articolul)\s+\d+',  # la art. 132
        r'din\s+(?:art\.|articolul)\s+\d+', # din art. 132
        r'de\s+(?:art\.|articolul)\s+\d+'
    ]

    text_lower = text_line.lower()
    for pattern in reference_patterns:
        if re.search(pattern, text_lower):
            return True
    return False

def extract_articles(text, doc_type):
    print(f"🔍 DEBUG pentru {doc_type}:")
    chunks = []
    
    ARTICLE_RE = re.compile(
        r'(?:^|\s)(?:ARTICOLUL|Articolul|ART\.?|Art\.)\s+(\d+(?:\^\d+)?)\.?\b'
        r'(?:\s*[-–]\s*)?(?:\s*\n?\s*(.+?))?(?=\s|$|\n)',
        flags=re.MULTILINE
    )
    
    ARTICLE_REFERENCE = re.compile(
        r'(?:conform|potrivit|prevederile|prevazut\s+(?:la|de)|dispozitiile|la|din)\s+(?:art\.|articolul)\s+\d+\b',
        flags=re.MULTILINE | re.IGNORECASE
    )
    
    all_matches = ARTICLE_RE.findall(text)
    print(f"   Total articole găsite cu ARTICLE_RE: {len(all_matches)}")
    if all_matches:
        print(f"   Primele 10: {all_matches[:10]}")
    
    # Gasește referințele pentru a le exclude ulterior
    references = ARTICLE_REFERENCE.findall(text)
    print(f"   Referințe găsite (de exclus): {len(references)}")
    if references:
        print(f"   Primele referințe: {references[:5]}")
    
    # Iterează prin toate meciurile găsite
    for match in ARTICLE_RE.finditer(text):
        full_match = match.group(0)
        article_num = match.group(1)
        article_title = match.group(2) if match.group(2) else ""
        
        #print(f"   🔍 Găsit: '{full_match.strip()}'")
        
        # Verificare mai riguroasă pentru referințe
        context_start = max(0, match.start() - 100)  # Context mai mare
        context_end = min(len(text), match.end() + 50)
        context = text[context_start:context_end]
        
        # Verificăm dacă articolul este într-un context de referință
        is_reference = False
        
        # 1. Verificăm contextul din jurul match-ului
        if ARTICLE_REFERENCE.search(context):
            is_reference = True
            
        # 2. Verificăm dacă linia întreagă pare a fi o referință
        line_start = text.rfind('\n', 0, match.start()) + 1
        line_end = text.find('\n', match.end())
        if line_end == -1:
            line_end = len(text)
        full_line = text[line_start:line_end]
        
        if is_article_reference(full_line):
            is_reference = True
            
        # 3. Verificăm dacă nu urmează conținut substanțial
        content_preview = text[match.end():match.end()+10].strip()
        if len(content_preview) < 10 and re.search(r'(?:art\.|articolul)\s+\d+', content_preview, re.IGNORECASE):
            is_reference = True
        
        if not is_reference:
            # Extragem conținutul articolului
            start_pos = match.end()
            
            # Găsim sfârșitul articolului (următorul articol sau sfârșitul textului)
            next_article = ARTICLE_RE.search(text, start_pos)
            if next_article:
                end_pos = next_article.start()
                content = text[start_pos:end_pos].strip()
            else:
                content = text[start_pos:].strip()
            
            # Curățăm conținutul de linii goale multiple
            content = re.sub(r'\n{3,}', '\n\n', content)
            content = content.strip()
            
            if content and len(content) > 10:  # Doar dacă avem conținut substanțial
                chunk = {
                    'id': f"{doc_type}_art_{article_num}",
                    'text': content,
                    'metadata': {
                        'doc_type': doc_type,
                        'article_number': article_num,
                        'article_title': article_title.strip() if article_title else ""
                    }
                }
                chunks.append(chunk)
                #print(f"   ✅ Articol {article_num} extras ({len(content)} caractere)")
            else:
                print(f"   ⚠️  Articol {article_num} - conținut prea scurt: '{content[:50]}'")
        #else:
            #print(f"   ✗ Exclus ca referință: 'Art. {article_num}'")
            #print(f"      Context: '{context[:20]}...'")
            #print(f"      Linie completă: '{full_line[:20]}...'")
    
    print(f"   🎯 FINAL: {len(chunks)} articole extrase din {doc_type}")
    return chunks

def process_pdf(pdf_path):

    print(f"🔍 Procesez: {pdf_path.name}")
    
    raw_text = extract_text_from_pdf(pdf_path)
    print(f"   Lungime text brut: {len(raw_text)} caractere")
    
    clean_text_content = clean_text(raw_text)
    print(f"   Lungime text curat: {len(clean_text_content)} caractere")
    
    doc_type = identify_doc_type(pdf_path.name)
    print(f"   Tip document: {doc_type}")
    
    chunks = extract_articles(clean_text_content, doc_type)
    print(f"   ✅ {len(chunks)} articole extrase")
    
    return chunks

def setup_embeddings():
    print("🔄 Configurare model embeddings...")

    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    test_embedding = model.encode(["test"], convert_to_numpy=True)
    print(f"✅ Model configurat dimensiune {test_embedding.shape[1]}")
    return model

def generate_embeddings(chunks, model, batch_size=32):    
    print(f"🔄 Generez embeddings pentru {len(chunks)} chunks...")

    texts = [chunk['text'] for chunk in chunks]
    embeddings = []
    
    for i in tqdm(range(0, len(chunks), batch_size)):
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
        embeddings.extend(batch_embeddings)
    
    for chunk, embedding in zip(chunks, embeddings):
        chunk['embedding'] = embedding
    
    print(f"✅ {len(embeddings)} embeddings generate")
    return chunks

def create_faiss_index(chunks_with_embeddings):

    embeddings = np.array([chunk['embedding'] for chunk in chunks_with_embeddings])
    dimension = embeddings.shape[1]
    
    index = faiss.IndexFlatL2(dimension)
    faiss.normalize_L2(embeddings)
    index.add(embeddings)
    
    print(f"✅ Index FAISS creat cu {index.ntotal} vectori de dimensiune {dimension}")
    return index

def save_processed_data(chunks, index, model, output_dir="processed"):
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    # 1.  Save embeddings + metadata pentru vector DB
    embeddings_data = []
    for i, chunk in enumerate(chunks):
        embeddings_data.append({
            'id': chunk['id'],
            'embedding': chunk['embedding'].tolist(),  # Convert numpy to list for JSON
            'metadata': chunk['metadata']
        })
    
    with open(output_path / "embeddings_with_metadata.json", "w", encoding="utf-8") as f:
        json.dump(embeddings_data, f, indent=2, ensure_ascii=False)
    
    # Debbug: Salvează textul brut al chunk-urilor
    texts_data = []
    for chunk in chunks:
        texts_data.append({
            'id': chunk['id'],
            'text': chunk['text'],
            'metadata': chunk['metadata']
        })
    
    with open(output_path / "chunks_text.json", "w", encoding="utf-8") as f:
        json.dump(texts_data, f, indent=2, ensure_ascii=False)
    
    # 3. Salvam index FAISS - necesar pentru căutare rapidă
    faiss.write_index(index, str(output_path / "faiss_index.bin"))
    
    # 4. Salvam embeddings ca numpy array - loading rapid 
    embeddings_array = np.array([chunk['embedding'] for chunk in chunks])
    np.save(output_path / "embeddings.npy", embeddings_array)
    
    print(f"💾 Vector DB assets salvate în {output_dir}:")
    print(f"   📁 embeddings_with_metadata.json - pentru vector DB")
    print(f"   📁 faiss_index.bin - pentru căutare")
    print(f"   📁 embeddings.npy - array numpy")
    print(f"   📁 chunks_text.json - texte pentru debug")

def load_processed_data(input_dir="processed"):

    input_path = Path(input_dir)
    
    with open(input_path / "embeddings_with_metadata.json", "r", encoding="utf-8") as f:
        embeddings_data = json.load(f)
    
    index = faiss.read_index(str(input_path / "faiss_index.bin"))
    
    with open(input_path / "chunks_text.json", "r", encoding="utf-8") as f:
        texts_data = json.load(f)
    
    print(f"✅ Date încărcate din {input_dir}")
    print(f"   📊 {len(embeddings_data)} embeddings")
    print(f"   🔍 Index FAISS cu {index.ntotal} vectori")
    
    return embeddings_data, index, texts_data

def show_statistics(chunks):
    print("\n📊 Statistici chunks:")
    print(f"Total chunks: {len(chunks)}")
    
    doc_stats = {}
    for chunk in chunks:
        doc_type = chunk['metadata']['doc_type']
        doc_stats[doc_type] = doc_stats.get(doc_type, 0) + 1
    
    for doc_type, count in sorted(doc_stats.items()):
        print(f"{doc_type}: {count} articole")

def show_failed_extractions(directory_path, chunks):

    directory = Path(directory_path)
    pdf_files = list(directory.glob("*.pdf"))
    
    # Colectează tipurile de documente care au avut extracții
    extracted_files = {}
    for chunk in chunks:
        doc_type = chunk['metadata']['doc_type']
        if doc_type not in extracted_files:
            extracted_files[doc_type] = []
        extracted_files[doc_type].append(chunk)
    
    failed_files = []
    
    print("\n❌ Analiză fișiere:")
    for pdf_file in pdf_files:
        doc_type = identify_doc_type(pdf_file.name)
        chunk_count = len(extracted_files.get(doc_type, []))
        
        if chunk_count == 0:
            failed_files.append(pdf_file)
            print(f"   ❌ {pdf_file.name} (tip: {doc_type}) - 0 articole")
        else:
            print(f"   ✅ {pdf_file.name} (tip: {doc_type}) - {chunk_count} articole")
    
    # Pentru fișierele failed, arată formatul textului
    if failed_files:
        print(f"\n🔍 ANALIZA FORMATELOR pentru {len(failed_files)} fișiere failed:")
        
        for pdf_file in failed_files:
            print(f"\n📄 === {pdf_file.name} ===")
            try:
                raw_text = extract_text_from_pdf(pdf_file)
                clean_text_content = clean_text(raw_text)
                                
                # Caută toate pattern-urile posibile
                patterns_to_check = [
                    (r'ARTICOLUL\s+\d+', 'ARTICOLUL num'),
                    (r'Articolul\s+\d+', 'Articolul num'),
                    (r'Art\.\s+\d+', 'Art. num'),
                    (r'ART\.\s+\d+', 'ART. num'),
                    (r'art\.\s+\d+', 'art. num'),
                    (r'Art\s+\d+', 'Art num (fără punct)'),
                    (r'ART\s+\d+', 'ART num (fără punct)'),
                    (r'Article\s+\d+', 'Article num (engleza)'),
                    (r'\d+\.\s+', 'num. (format simplu)'),
                ]
                
                print(f"🔍 Pattern-uri găsite:")
                found_any = False
                for pattern, description in patterns_to_check:
                    matches = re.findall(pattern, clean_text_content, re.MULTILINE | re.IGNORECASE)
                    if matches:
                        print(f"   ✓ {description}: {len(matches)} găsite - {matches[:5]}")
                        found_any = True
                
                if not found_any:
                    print("   ❌ Niciun pattern de articol găsit!")
                
                # Caută linii care încep cu numere
                lines_with_numbers = []
                for line_num, line in enumerate(clean_text_content.split('\n')[:100]):
                    if re.match(r'^\s*\d+', line.strip()):
                        lines_with_numbers.append(f"Linia {line_num}: '{line.strip()}'")
                
                if lines_with_numbers:
                    print(f"📊 Primele linii care încep cu numere:")
                    for line_info in lines_with_numbers[:10]:
                        print(f"   {line_info}")
                        
            except Exception as e:
                print(f"   ❌ Eroare la procesarea {pdf_file.name}: {e}")
    else:
        print("✅ Toate fișierele au fost procesate cu succes!")

def search_chunks(query, chunks, index, model, top_k=5):
    """Caută în chunks folosind similaritatea semantică"""

    query_embedding = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)
    
    distances, indices = index.search(query_embedding, top_k)
    
    results = []
    for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
        if idx < len(chunks):
            results.append({
                'rank': i + 1,
                'score': 1 - distance,  
                'chunk': chunks[idx]
            })
    
    return results

def demo_search(chunks, index, model):

    test_queries = [
        "contractul de muncă",
        "drepturile salariatului",
        "concediul de odihnă",
        "salariul minim",
        "întreruperea contractului de muncă"
    ]
    
    print("\n🔍 Demo căutare:")
    for query in test_queries:
        print(f"\n❓ Întrebare: '{query}'")
        results = search_chunks(query, chunks, index, model, top_k=3)
        
        for result in results:
            chunk = result['chunk']
            print(f"   {result['rank']}. [{chunk['id']}] Score: {result['score']:.3f}")
            print(f"      {chunk['text'][:100]}...")

def process_directory(directory_path):
    print(f"🗂️  Procesez directorul: {directory_path}")

    directory = Path(directory_path)
    pdf_files = list(directory.glob("*.pdf"))
    all_chunks = []

    print(f"   Găsite {len(pdf_files)} fișiere PDF")
    
    for pdf_file in pdf_files:
        chunks = process_pdf(pdf_file)
        all_chunks.extend(chunks)
    
    print(f"\n✅ Total chunks create: {len(all_chunks)}")
    return all_chunks

def main():
    directory_path = "dataset"
    
    chunks = process_directory(directory_path)
    show_statistics(chunks)
    show_failed_extractions(directory_path, chunks)
    
    model = setup_embeddings()    
    chunks_with_embeddings = generate_embeddings(chunks, model)
    index = create_faiss_index(chunks_with_embeddings)
    
    save_processed_data(chunks_with_embeddings, index, model)
    demo_search(chunks_with_embeddings, index, model)

if __name__ == "__main__":
    main()

🗂️  Procesez directorul: dataset
   Găsite 6 fișiere PDF
🔍 Procesez: COD FISCAL (A) 08_09_2015.pdf
   Lungime text brut: 2448117 caractere
   Lungime text curat: 2415628 caractere
   Tip document: CF
🔍 DEBUG pentru CF:
   Total articole găsite cu ARTICLE_RE: 2231
   Primele 10: [('1', 'Scopul'), ('2', 'Impozitele,'), ('2', ','), ('2', ','), ('2', ','), ('3', 'Principiile'), ('4', 'Modificarea'), ('4', ','), ('5', 'Norme'), ('6', 'Aplicarea')]
   Referințe găsite (de exclus): 2539
   Primele referințe: ['prevederile art. 157', 'prevederile art. 41', 'la art. 16', 'la art. 2', 'din Articolul 2']
   🎯 FINAL: 1559 articole extrase din CF
   ✅ 1559 articole extrase
🔍 Procesez: codul-muncii.pdf
   Lungime text brut: 220710 caractere
   Lungime text curat: 218786 caractere
   Tip document: CM
🔍 DEBUG pentru CM:
   Total articole găsite cu ARTICLE_RE: 293
   Primele 10: [('1', '(1)'), ('2', 'Dispozitiile'), ('3', '(1)'), ('4', '(1)'), ('5', '(1)'), ('6', '(1)'), ('7', 'Salariatii'), ('8', '(1)

100%|██████████| 143/143 [10:48<00:00,  4.54s/it]


✅ 4558 embeddings generate
✅ Index FAISS creat cu 4558 vectori de dimensiune 768
💾 Vector DB assets salvate în processed:
   📁 embeddings_with_metadata.json - pentru vector DB
   📁 faiss_index.bin - pentru căutare
   📁 embeddings.npy - array numpy
   📁 chunks_text.json - texte pentru debug

🔍 Demo căutare:

❓ Întrebare: 'contractul de muncă'
   1. [CM_art_37] Score: 0.574
      si obligatiile privind relatiile de munca dintre angajator si salariat se stabilesc potrivit legii, ...
   2. [CM_art_21] Score: 0.536
      La incheierea contractului individual de munca sau pe parcursul executarii acestuia, partile pot neg...
   3. [CM_art_16] Score: 0.525
      Contractul individual de munca se incheie in baza consimtamantului partilor, in forma scrisa, in lim...

❓ Întrebare: 'drepturile salariatului'
   1. [CM_art_39] Score: 0.738
      Salariatul are, in principal, urmatoarele drepturi: 
a) dreptul la salarizare pentru munca depusa; 
...
   2. [CM_art_47] Score: 0.714
      Drepturile cuve